In [1]:
import os

In [2]:
os.chdir('../')
%pwd

'/home/paladin/Downloads/Facial_Impression_Recognition_Calassification'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    ckeckpoint_model_filepath: Path

In [8]:

from cnnClassifier.constants import *
from cnnClassifier.utils import read_yaml, create_directories


class configurationManeger:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 secret_filepath = SECRET_FILE_PATH,                 
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath) 
        self.secret = read_yaml(secret_filepath)        
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])   
    
    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.ckeckpoint_model_filepath)

        create_directories([config.tensorboard_root_log_dir, model_ckpt_dir ])

        prepare_callbacks_config = PrepareCallbacksConfig(
           root_dir= config.root_dir,
           tensorboard_root_log_dir= config.tensorboard_root_log_dir,
           ckeckpoint_model_filepath=  config.ckeckpoint_model_filepath
        )

        return prepare_callbacks_config 


In [10]:
import sys
from cnnClassifier.exception import CustomException
import keras
import time

In [11]:
class PrepareCallbacks:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    #Enable visualizations for TensorBoard.
    @property
    def _create_tb_callbacks(self): 
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
            )   
        return keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
        
    #ModelCheckpoint callback is used in conjunction with training using model. fit() to save a model or weights (in a checkpoint file) //
    # at some interval, so the model or weights can be loaded later to continue the training from the state saved.    
    @property
    def _create_ckpt_callbacks(self):
        return keras.callbacks.ModelCheckpoint(
                filepath = self.config.checkpoint_model_filepath,
                save_best_only = True)


    def get_tb_ckpt_callbacks(self):
        return [self._create_tb_callbacks, self._create_ckpt_callbacks]

In [12]:
try:
    config = configurationManeger()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallbacks(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
except Exception as e:
    raise CustomException(e, sys)


NameError: name 'CustomException' is not defined